In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
%load_ext autoreload

%autoreload 2
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

This is a kaggle data set. The images are high resolution chest x-ray images. 

Class descriptions
There are 15 classes (14 diseases, and one for "No findings"). Images can be classified as "No findings" or one or more disease classes:

Atelectasis
Consolidation
Infiltration
Pneumothorax
Edema
Emphysema
Fibrosis
Effusion
Pneumonia
Pleural_thickening
Cardiomegaly
Nodule Mass
Hernia

62 thousand imagesare used are used in this notebook.


In [ ]:
dir =  "/kaggle/input/data"
train = os.listdir("/kaggle/input/data")

In [ ]:
image1 =  os.path.join(dir + "images_001")

In [ ]:
from glob import glob 
import pandas as pd

dataset = pd.read_csv('../input/data/Data_Entry_2017.csv')
paths = {os.path.basename(x): x for x in 
                   glob(os.path.join('..', 'input', 'data',  'images*', '*', '*.png'))}
print('Scans found:', len(paths), ', Total Headers', dataset.shape[0])
dataset['path'] = dataset['Image Index'].map(paths.get)
dataset['Cardiomegaly'] = dataset['Finding Labels'].map(lambda x: 'Cardiomegaly' in x)
dataset['Finding'] = dataset['Finding Labels'].map(lambda x: x != 'No Finding').astype(int)
dataset['Patient Age'] = np.clip(dataset['Patient Age'], 5, 100)
label_freq = dataset['Finding Labels'].apply(lambda s: str(s).split('|')).explode().value_counts().sort_values(ascending=False)
rare = list(label_freq[label_freq<10].index)
dataset['labels'] = dataset['Finding Labels'].apply(lambda s: [l for l in str(s).split('|') if l not in rare])



In [ ]:
#dataset['Finding Labels'] = dataset['Finding Labels'].map(lambda x: x.replace('No Finding', 'n'))
from itertools import chain
all_labels = np.unique(list(chain(*dataset['Finding Labels'].map(lambda x: x.split('|')).tolist())))
all_labels = [x for x in all_labels if len(x)>0]
print('All Labels ({}): {}'.format(len(all_labels), all_labels))
for c_label in all_labels:
    if len(c_label)>1: # leave out empty labels
        dataset[c_label] = dataset['Finding Labels'].map(lambda finding: 1.0 if c_label in finding else 0)
#dataset.sample(3)
dataset['disease_vec'] = dataset.apply(lambda x: [x[all_labels].values], 1).map(lambda x: x[0])

In [ ]:
data = dataset[['Image Index' , 'Patient ID', 'path', 'Finding' , 'labels' , 'disease_vec']]

In [ ]:
noFind = data[data['Finding']==0]['Image Index'].tolist()
Find = data[data['Finding']==1]['Image Index'].tolist()
noFind_path = data[data['Finding']==0]['path'].tolist()
Find_path = data[data['Finding']==1]['path'].tolist()

In [ ]:
path = (glob(os.path.join('..' , 'input' , 'data' , 'images*' , '*' , '*' )))

In [ ]:
# #noFind_paths = []
# #for i in noFind[:10000]:
#     destination = (glob(os.path.join('..' , 'input' , 'data' , 'images*' , '*' , i )))
#     noFind_paths.append(destination)

In [ ]:
# Find_paths = []
# for i in Find:
#     destination = (glob(os.path.join('..' , 'input' , 'data' , 'images*' , '*' , i )))
#     Find_paths.append(destination)

In [ ]:
final_paths = Find_path + noFind_path[:10000]

final_data = data[data['Finding']==1]
temp= data[data['Finding']==0][:10000]


final_data.append(temp)
final_paths = final_data['path'].tolist()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
train_df, valid_df , train_y , valid_y = train_test_split(final_paths, 
                                                          final_data['disease_vec'].tolist(),
                                   test_size = 0.6, 
                                   random_state = 2000,
                                   #stratify = data['labels'].map(lambda x: x[:4]
                                     )

valid_df, test_df , valid_y , test_y = train_test_split(valid_df, 
                                                        valid_y,
                                   test_size = 0.5, 
                                   random_state = 2000)



def flow_from_dataframe(img_data_gen, in_df, path_col, y_col, **dflow_args):
    base_dir = os.path.dirname(in_df[path_col].values[0])
    print('## Ignore next message from keras, values are replaced anyways')
    df_gen = img_data_gen.flow_from_directory(base_dir, 
                                     class_mode = 'sparse',
                                    **dflow_args)
    df_gen.filenames = in_df[path_col].values
    df_gen.classes = np.stack(in_df[y_col].values)
    df_gen.samples = in_df.shape[0]
    df_gen.n = in_df.shape[0]
    df_gen._set_index_array()
    df_gen.directory = '' # since we have the full path
    print('Reinserting dataframe: {} images'.format(in_df.shape[0]))
    return df_gen





In [44]:
SHUFFLE_BUFFER_SIZE = 1024
BATCH_SIZE = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE 
CHANNELS = 3
IMG_SIZE = 128

def parse_function(filename, label):
    image_string = tf.io.read_file(filename)
    image_decoded = tf.image.decode_jpeg(image_string, channels=CHANNELS)
    image_resized = tf.image.resize(image_decoded, [IMG_SIZE, IMG_SIZE])
    image_normalized = image_resized / 255.0
    return image_normalized, label



In [ ]:


def create_dataset(filenames, labels, is_training=True):

    dataset = tf.data.Dataset.from_tensor_slices((filenames, labels))
    dataset = dataset.map(parse_function, num_parallel_calls=AUTOTUNE)
    
    if is_training == True:
        dataset = dataset.cache()
        dataset = dataset.shuffle(buffer_size=SHUFFLE_BUFFER_SIZE)
        
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(buffer_size=AUTOTUNE)
    
    return dataset



In [ ]:
LR = 1e-5 
EPOCHS = 1

In [ ]:
# WEIGHTS_PATH = ('https://github.com/fchollet/deep-learning-models/'
#                 'releases/download/v0.2/'
#                 'resnet50_weights_tf_dim_ordering_tf_kernels.h5')
# WEIGHTS_PATH_NO_TOP = ('https://github.com/fchollet/deep-learning-models/'
#                        'releases/download/v0.2/'
#                        'resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5')

IMG_SIZE = (128,128)

datagen_train = ImageDataGenerator(rescale = 1./255,
                                featurewise_center=True,
                                featurewise_std_normalization=True,
                                rotation_range=20,
                                width_shift_range=0.2,
                                height_shift_range=0.2,
                                horizontal_flip=False,
                                shear_range=0.2,
                                zoom_range=0.2,
                                fill_mode = 'nearest')

datagen_valid = ImageDataGenerator(rescale = 1./255)
datagen_test = ImageDataGenerator(rescale = 1./255)


In [ ]:
# import tensorflow as tf
# base_model = tf.keras.applications.ResNet50(input_shape=(128,128,3),
#                                                include_top=False,
#                                                weights='imagenet')

In [ ]:
# last_layer=base_model.get_layer('conv3_block4_out').output
# x=tf.keras.layers.Flatten()(last_layer)
# # x= tf.keras.layers.Dense(16000 , activation = 'relu')(x)
# x= tf.keras.layers.Dense(3200, activation = 'relu')(x)
# x= tf.keras.layers.Dense(512,activation = 'relu')(x)
# x= tf.keras.layers.Dense(64 ,activation = 'relu')(x)
# x= tf.keras.layers.Dense(15 ,activation = 'relu')(x)

# model = tf.keras.Model(base_model.input , x)


In [ ]:
# model.summary()

In [ ]:
# # base_model.summary()
# for layer in model.layers:
#     layer.trainable=False
#     if layer.name[-2:]=='bn' or layer.name[:5]=='conv3' or layer.name[:1]=='f' or layer.name[:1]=='d':
#         layer.trainable=True

In [ ]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

In [ ]:
# with tf.device('/gpu:1')

In [ ]:
train_ds = create_dataset(train_df, train_y)
val_ds = create_dataset(valid_df, valid_y)


In [ ]:
@tf.function
def macro_soft_f1(y, y_hat):
    y = tf.cast(y, tf.float32)
    y_hat = tf.cast(y_hat, tf.float32)
    tp = tf.reduce_sum(y_hat * y, axis=0)
    fp = tf.reduce_sum(y_hat * (1 - y), axis=0)
    fn = tf.reduce_sum((1 - y_hat) * y, axis=0)
    soft_f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    cost = 1 - soft_f1 # reduce 1 - soft-f1 in order to increase soft-f1
    macro_cost = tf.reduce_mean(cost) # average on all labels
    return macro_cost


In [ ]:
@tf.function
def macro_f1(y, y_hat, thresh=0.5):
    """Compute the macro F1-score on a batch of observations (average F1 across labels)
    
    Args:
        y (int32 Tensor): labels array of shape (BATCH_SIZE, N_LABELS)
        y_hat (float32 Tensor): probability matrix from forward propagation of shape (BATCH_SIZE, N_LABELS)
        thresh: probability value above which we predict positive
        
    Returns:
        macro_f1 (scalar Tensor): value of macro F1 for the batch
    """
    y_pred = tf.cast(tf.greater(y_hat, thresh), tf.float32)
    tp = tf.cast(tf.math.count_nonzero(y_pred * y, axis=0), tf.float32)
    fp = tf.cast(tf.math.count_nonzero(y_pred * (1 - y), axis=0), tf.float32)
    fn = tf.cast(tf.math.count_nonzero((1 - y_pred) * y, axis=0), tf.float32)
    f1 = 2*tp / (2*tp + fn + fp + 1e-16)
    macro_f1 = tf.reduce_mean(f1)
    return macro_f1

In [ ]:
# model.compile(
#   optimizer=tf.keras.optimizers.Adam(learning_rate=LR),
#   loss=macro_soft_f1,
#   metrics=[macro_f1])

In [ ]:
# import matplotlib.pyplot as plt
# import matplotlib.image as mping
# img = mping.imread(final_paths[0])
# plt.imshow(img)
# model.summary()

In [ ]:
# history = model.fit(train_ds,
#                     epochs=EPOCHS,
#                     validation_data=create_dataset(valid_df , valid_y))

In [ ]:
# model.summary()

In [ ]:
# from tensorflow.keras.optimizers import RMSprop
# model.compile(optimizer = RMSprop(lr= 0.001) ,
#              loss=macro_soft_f1,
#             metrics=[macro_f1])

In [ ]:
# print(datagen_valid)

history = model.fit_generator(datagen_train,
                              validation_data=datagen_valid,
                              steps_per_epoch=200,
                              epochs=10,
                              validation_steps=10,
                              verbose=2)

In [46]:
LR=0.0001

In [ ]:
# with tf.device("/gpu:0"):
#     train_ds = create_dataset(train_df, train_y)
#     val_ds = create_dataset(valid_df, valid_y)
base_model = tf.keras.applications.ResNet50(input_shape=(128,128,3),
                                       include_top=False,
                                       weights='imagenet')
last_layer=base_model.get_layer('conv3_block4_out').output
x=tf.keras.layers.Flatten()(last_layer)
#     x= tf.keras.layers.Dense(3200, activation = 'relu')(x)
x= tf.keras.layers.Dense(512,activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
x= tf.keras.layers.Dense(64 ,activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x= tf.keras.layers.Dense(15 ,activation = 'relu')(x)
model = tf.keras.Model(base_model.input , x)
# for layer in model.layers:
#     layer.trainable=False
#     if layer.name[-2:]=='bn' or layer.name[:5]=='conv3' or layer.name[:1]=='f' or layer.name[:1]=='d':
#         layer.trainable=True
from tensorflow.keras.optimizers import RMSprop        
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),loss=macro_soft_f1,metrics=[macro_f1,'acc'])  
history = model.fit(train_ds,
            epochs=EPOCHS,
            validation_data=create_dataset(valid_df , valid_y))

In [47]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LR),loss=macro_soft_f1,metrics=[macro_f1,'acc']) 

In [42]:
history = model.fit(train_ds,
            epochs=5,
            validation_data=create_dataset(valid_df , valid_y))

Train for 81 steps, validate for 61 steps
Epoch 1/5
81/81 [==============================] - 189s 2s/step - loss: 0.7625 - macro_f1: 0.0671 - acc: 0.0341 - val_loss: 0.9093 - val_macro_f1: 0.0357 - val_acc: 0.0097
Epoch 2/5
81/81 [==============================] - 28s 350ms/step - loss: 0.7030 - macro_f1: 0.0949 - acc: 0.0615 - val_loss: 0.8030 - val_macro_f1: 0.0670 - val_acc: 0.0239
Epoch 3/5
81/81 [==============================] - 28s 348ms/step - loss: 0.7107 - macro_f1: 0.0900 - acc: 0.0511 - val_loss: 0.9047 - val_macro_f1: 0.0388 - val_acc: 0.0344
Epoch 4/5
81/81 [==============================] - 28s 349ms/step - loss: 0.7579 - macro_f1: 0.0786 - acc: 0.0547 - val_loss: 0.7847 - val_macro_f1: 0.0612 - val_acc: 0.0321
Epoch 5/5
81/81 [==============================] - 28s 347ms/step - loss: 0.7385 - macro_f1: 0.0742 - acc: 0.0295 - val_loss: 0.8026 - val_macro_f1: 0.0865 - val_acc: 0.0527


In [43]:
history1 = model.fit(train_ds,
            epochs=20,
            validation_data=create_dataset(valid_df , valid_y))

Train for 81 steps, validate for 61 steps
Epoch 1/20
81/81 [==============================] - 183s 2s/step - loss: 0.7553 - macro_f1: 0.0720 - acc: 0.0351 - val_loss: 0.9108 - val_macro_f1: 0.0444 - val_acc: 0.0375
Epoch 2/20
81/81 [==============================] - 28s 346ms/step - loss: 0.8820 - macro_f1: 0.0393 - acc: 0.0429 - val_loss: 0.8694 - val_macro_f1: 0.0271 - val_acc: 0.0092
Epoch 3/20
81/81 [==============================] - 28s 347ms/step - loss: 0.8460 - macro_f1: 0.0350 - acc: 0.0394 - val_loss: 0.8591 - val_macro_f1: 0.0209 - val_acc: 0.0188
Epoch 4/20
81/81 [==============================] - 28s 345ms/step - loss: 0.8386 - macro_f1: 0.0402 - acc: 0.0390 - val_loss: 0.8541 - val_macro_f1: 0.0254 - val_acc: 0.0231
Epoch 5/20
81/81 [==============================] - 28s 350ms/step - loss: 0.8402 - macro_f1: 0.0324 - acc: 0.0740 - val_loss: 0.9266 - val_macro_f1: 0.0274 - val_acc: 0.0044
Epoch 6/20
81/81 [==============================] - 28s 346ms/step - loss: 0.8391 - m

In [ ]:
history3=model.fit(train_ds,
            epochs=10,
            validation_data=create_dataset(valid_df , valid_y))

Train for 81 steps, validate for 486 steps
Epoch 1/10
81/81 [==============================] - 188s 2s/step - loss: 0.7621 - macro_f1: 0.0643 - acc: 0.1442 - val_loss: 0.8202 - val_macro_f1: 0.0527 - val_acc: 0.1812
Epoch 2/10
81/81 [==============================] - 31s 381ms/step - loss: 0.7428 - macro_f1: 0.0606 - acc: 0.1437 - val_loss: 0.8118 - val_macro_f1: 0.0503 - val_acc: 0.1716
Epoch 3/10
81/81 [==============================] - 31s 381ms/step - loss: 0.7307 - macro_f1: 0.0561 - acc: 0.1444 - val_loss: 0.8159 - val_macro_f1: 0.0422 - val_acc: 0.1658
Epoch 4/10
81/81 [==============================] - 31s 380ms/step - loss: 0.7225 - macro_f1: 0.0520 - acc: 0.1423 - val_loss: 0.8169 - val_macro_f1: 0.0400 - val_acc: 0.1580
Epoch 5/10
81/81 [==============================] - 31s 381ms/step - loss: 0.7187 - macro_f1: 0.0468 - acc: 0.1388 - val_loss: 0.8370 - val_macro_f1: 0.0326 - val_acc: 0.1616
Epoch 6/10
81/81 [==============================] - 31s 381ms/step - loss: 0.7174 - 